# Coursework 1: ML basics and fully-connected networks

#### Instructions

Please submit a version of this notebook containing your answers on CATe as *CW1*. Write your answers in the cells below each question.

We recommend that you work on the Ubuntu workstations in the lab. This assignment and all code were only tested to work on these machines. In particular, we cannot guarantee compatibility with Windows machines and cannot promise support if you choose to work on a Windows machine.

You can work from home and use the lab workstations via ssh (for list of machines: https://www.doc.ic.ac.uk/csg/facilities/lab/workstations). 

Once logged in, run the following commands in the terminal to set up a Python environment with all the packages you will need.

    export PYTHONUSERBASE=/vol/bitbucket/nuric/pypi
    export PATH=/vol/bitbucket/nuric/pypi/bin:$PATH

Add the above lines to your `.bashrc` to have these enviroment variables set automatically each time you open your bash terminal.

Any code that you submit will be expected to run in this environment. Marks will be deducted for code that fails to run.

Run `jupyter-notebook` in the coursework directory to launch Jupyter notebook in your default browser.

DO NOT attempt to create a virtualenv in your home folder as you will likely exceed your file quota.

**DEADLINE: 7pm, Tuesday 5th February, 2019**

## Part 1

1. Describe two practical methods used to estimate a supervised learning model's performance on unseen data. Which strategy is most commonly used in most deep learning applications, and why?
2. Suppose that you have reason to believe that your multi-layer fully-connected neural network is overfitting. List four things that you could try to improve generalization performance.

## Ans

#### Q1
Since the model has been acquired, we can generate new data to validate or use existing data to apply cross validation. The cross validation is more commonly used, because it is usually costly to acquire new data, and by segmenting training data into several part and use one part each time as validation set, we can actually test performance on the whole training data.
#### Q2.
1. Add coefficient penalizer as regularization
2. Add an upper bound to largest possible coefficient
3. Add drop out
4. Use cross-validation in traning process


## Part 2

1. Why can gradient-based learning be difficult when using the sigmoid or hyperbolic tangent functions as hidden unit activation functions in deep, fully-connected neural networks?
2. Why is the issue that arises in the previous question less of an issue when using such functions as output unit activation functions, provided that an appropriate loss function is used?
3. What would happen if you initialize all the weights to zero in a multi-layer fully-connected neural network and attempt to train your model using gradient descent? What would happen if you did the same thing for a logistic regression model?

## Ans

#### Q1
It is easy to cause gradient vanishing, since when requesting contribution of gradient of each layer to the final outcome, each gradient of weight will multiply gradients of layers after it, and approach to zero if some of gradients after it are small. As a result, any of activation function that have a range of points on which the gradient is approaching to zero will have difficulty on gradient base learning in hidden layer.

#### Q2.
Similar as previous question, the calculation of gradient of weight on output unit has no dependncy on previous layers, and therefore will less likely to cause gradient vanishing.

#### Q3.
The neural network will be useless, since all weights are zero, weights from same layer have same performance, same back propagation gradients, and can not be distinguished and trained. It would be fine if a logistic regression model initialise weights to be zero.


## Part 3

In this part, you will use PyTorch to implement and train a multinomial logistic regression model to classify MNIST digits.

Restrictions:
* You must use (but not modify) the code provided in `utils.py`. **This file is deliberately not documented**; read it carefully as you will need to understand what it does to complete the tasks.
* You are NOT allowed to use the `torch.nn` module.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultinomialLogisticRegressionClassifier` class below by filling in the missing parts (expected behaviour is prescribed in the documentation):
    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 92% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.


In [1]:
from utils import *

In [2]:
# *CODE FOR PART 3.1 IN THIS CELL*

class MultinomialLogisticRegressionClassifier:

    def __init__(self, weight_init_sd=100.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        self.weight_init_sd = weight_init_sd

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################

        layer_0_num = 784
        layer_1_num = 10
        
        weight_0 = torch.normal(mean = 0., std = self.weight_init_sd * torch.ones(layer_0_num, layer_1_num))
        weight_0.requires_grad_(requires_grad=True)

        self.weight = []
        self.weight.append(weight_0)
            
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be a Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################

        inputs = inputs.reshape(inputs.size()[0], 784)
        h = inputs.matmul(self.weight[0])
        h = F.log_softmax(h, dim=1)
        
        return h
        
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        
        return self.weight
    
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        
        weight_copy = self.weight[0]
        return abs(weight_copy.detach().numpy()).sum()
        
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 norm of the model's weight vector (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        
        weight_copy = self.weight[0]
        return pow(weight_copy.detach().numpy(), 2).sum()
        
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [3]:
# *CODE FOR PART 3.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *
model = MultinomialLogisticRegressionClassifier(weight_init_sd=.1)
res = run_experiment(
    model,
    optimizer=optim.SGD(model.parameters(), lr=.1, momentum=.2),
    # optimizer=optim.Adam(model.parameters(), lr=.005, betas=(.8, .9)),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=.2,
    l2_penalty_coef=.1,
    suppress_output=False
)

Epoch 0: training...
Train set:	Average loss: 0.4905, Accuracy: 0.8629
Validation set:	Average loss: 0.3636, Accuracy: 0.8968

Epoch 1: training...
Train set:	Average loss: 0.3437, Accuracy: 0.9029
Validation set:	Average loss: 0.3298, Accuracy: 0.9053

Epoch 2: training...
Train set:	Average loss: 0.3212, Accuracy: 0.9094
Validation set:	Average loss: 0.3211, Accuracy: 0.9122

Epoch 3: training...
Train set:	Average loss: 0.3094, Accuracy: 0.9133
Validation set:	Average loss: 0.3068, Accuracy: 0.9128

Epoch 4: training...
Train set:	Average loss: 0.3021, Accuracy: 0.9159
Validation set:	Average loss: 0.3027, Accuracy: 0.9173

Epoch 5: training...
Train set:	Average loss: 0.2964, Accuracy: 0.9176
Validation set:	Average loss: 0.2985, Accuracy: 0.9157

Epoch 6: training...
Train set:	Average loss: 0.2922, Accuracy: 0.9189
Validation set:	Average loss: 0.2968, Accuracy: 0.9167

Epoch 7: training...
Train set:	Average loss: 0.2886, Accuracy: 0.9201
Validation set:	Average loss: 0.2934, Ac

## Part 4

In this part, you will use PyTorch to implement and train a multi-layer fully-connected neural network to classify MNIST digits.

Your network must have three hidden layers with 128, 64, and 32 hidden units respectively.

The same restrictions as in Part 3 apply.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultilayerClassifier` class below by filling in the missing parts of the following methods (expected behaviour is prescribed in the documentation):

    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 97% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.

3. Describe an alternative strategy for initializing weights that may perform better than the strategy we have used here.

In [4]:
# *CODE FOR PART 4.1 IN THIS CELL*

class MultilayerClassifier:
    def __init__(self, activation_fun="sigmoid", weight_init_sd=1.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        super().__init__()
        self.activation_fun = activation_fun
        self.weight_init_sd = weight_init_sd

        if self.activation_fun == "relu":
            self.activation = F.relu
        elif self.activation_fun == "sigmoid":
            self.activation = torch.sigmoid
        elif self.activation_fun == "tanh":
            self.activation = torch.tanh
        else:
            raise NotImplementedError()

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        
        layer_0_num = 784
        layer_1_num = 128
        layer_2_num = 64
        layer_3_num = 32
        layer_4_num = 10
        
        weight_0 = torch.normal(mean = 0., std = self.weight_init_sd * torch.ones(layer_0_num, layer_1_num))
        weight_1 = torch.normal(mean = 0., std = self.weight_init_sd * torch.ones(layer_1_num, layer_2_num))
        weight_2 = torch.normal(mean = 0., std = self.weight_init_sd * torch.ones(layer_2_num, layer_3_num))
        weight_3 = torch.normal(mean = 0., std = self.weight_init_sd * torch.ones(layer_3_num, layer_4_num))
        
        weight_0.requires_grad_(requires_grad=True)
        weight_1.requires_grad_(requires_grad=True)
        weight_2.requires_grad_(requires_grad=True)
        weight_3.requires_grad_(requires_grad=True)

        self.weight = []
        self.weight.append(weight_0)
        self.weight.append(weight_1)
        self.weight.append(weight_2)
        self.weight.append(weight_3)
        
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        
        inputs = inputs.reshape(inputs.size()[0], 784)
        h = inputs.matmul(self.weight[0])
        h = self.activation(h)
        
        h = h.matmul(self.weight[1])
        h = self.activation(h)
        
        h = h.matmul(self.weight[2])
        h = self.activation(h)
        
        h = h.matmul(self.weight[3])
        h = F.log_softmax(h, dim=1)
        return h
        
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        
        return self.weight
        
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        
        l1 = 0
        
        for i in range(len(self.weight)):
            weight_copy = self.weight[i]
            l1 += abs(weight_copy.detach().numpy()).sum()
    
        return l1
    
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 norm of the model's weight vector (i.e. 
        square root of sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        
        l2 = 0
        
        for i in range(len(self.weight)):
            weight_copy = self.weight[i]
            l2 += pow(weight_copy.detach().numpy(), 2).sum()
    
        return l2
    
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [7]:
# *CODE FOR PART 4.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *

model = MultilayerClassifier(activation_fun='relu', weight_init_sd=.1)
res = run_experiment(
    model,
    optimizer=optim.Adam(model.parameters(), lr=.0005, betas=(.8, .9)),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=.01,
    l2_penalty_coef=.005,
    suppress_output=False
)

Epoch 0: training...
Train set:	Average loss: 0.4662, Accuracy: 0.8658
Validation set:	Average loss: 0.2303, Accuracy: 0.9320

Epoch 1: training...
Train set:	Average loss: 0.1857, Accuracy: 0.9454
Validation set:	Average loss: 0.1563, Accuracy: 0.9518

Epoch 2: training...
Train set:	Average loss: 0.1341, Accuracy: 0.9607
Validation set:	Average loss: 0.1319, Accuracy: 0.9585

Epoch 3: training...
Train set:	Average loss: 0.1059, Accuracy: 0.9685
Validation set:	Average loss: 0.1080, Accuracy: 0.9672

Epoch 4: training...
Train set:	Average loss: 0.0876, Accuracy: 0.9743
Validation set:	Average loss: 0.1043, Accuracy: 0.9673

Epoch 5: training...
Train set:	Average loss: 0.0738, Accuracy: 0.9779
Validation set:	Average loss: 0.1012, Accuracy: 0.9692

Epoch 6: training...
Train set:	Average loss: 0.0621, Accuracy: 0.9815
Validation set:	Average loss: 0.0977, Accuracy: 0.9715

Epoch 7: training...
Train set:	Average loss: 0.0539, Accuracy: 0.9842
Validation set:	Average loss: 0.0957, Ac

## Ans

For multi-layer neuron network,Same weight variance between each layer can increase the convergence speed. Therefore when initialising weight, it would be better if weights between different layers have a standard weight variance. Therefore, after initialising weight of first layer, we can use its variance as standard to initialise the rest of weights.
